In [ ]:
# !pip install requests pandas scikit-learn umap-learn plotly numpy

In [79]:
import requests
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import umap.umap_ as umap
import plotly.express as px

In [ ]:
# Ta clé API TMDb ici
API_KEY = "Clé_api"
BASE_URL = "https://api.themoviedb.org/3"

In [81]:

def fetch_movies(endpoint, params):
    url = f"{BASE_URL}/{endpoint}"
    params["api_key"] = API_KEY
    response = requests.get(url, params=params)
    response.raise_for_status()
    return response.json()

def get_popular_movies(pages=5, min_vote_count=1000):
    all_movies = []
    for page in range(1, pages + 1):
        data = fetch_movies("discover/movie", {
            "language": "fr-FR",
            "sort_by": "vote_average.desc",
            "vote_count.gte": min_vote_count,
            "page": page
        })
        all_movies.extend(data["results"])
    return all_movies

def get_recent_movies(pages=5):
    from datetime import datetime
    year = datetime.now().year
    all_movies = []
    for page in range(1, pages+1):
        data = fetch_movies("movie/popular", {
            "language": "fr-FR",
            "sort_by": "release_date.desc",
            "primary_release_year": year,
            "page": page
        })
        all_movies.extend(data["results"])
    return all_movies

def extract_movie_features(movies):
    # Récupère les colonnes utiles
    df = pd.DataFrame(movies)
    df = df[["id","title","overview","release_date","vote_average"]]
    df["overview"] = df["overview"].fillna("")
    df["release_date"] = pd.to_datetime(df["release_date"], errors="coerce")
    return df

In [82]:
# Récupération des données
popular_movies = get_popular_movies()
recent_movies = get_recent_movies()

df_popular = extract_movie_features(popular_movies)
df_recent = extract_movie_features(recent_movies)

# Ajout d'une colonne pour la source
df_popular["category"] = "popular"
df_recent["category"] = "recent"

In [83]:
df_popular

,id,title,overview,release_date,vote_average,category
0,278,Les Évadés,"En 1947, Andy Dufresne, un jeune banquier, est...",1994-09-23,8.700,popular
1,238,Le Parrain,La Seconde Guerre mondiale vient de s'achever....,1972-03-14,8.687,popular
2,240,"Le Parrain, 2e partie","Depuis la mort de Don Vito Corleone, son fils ...",1974-12-20,8.571,popular
3,424,La Liste de Schindler,Évocation des années de guerre d’Oskar Schindl...,1993-12-15,8.565,popular
4,389,Douze Hommes en colère,Un jeune homme d'origine modeste est accusé du...,1957-04-10,8.549,popular
...,...,...,...,...,...,...
95,11324,Shutter Island,"En 1954, le marshal Teddy Daniels et son coéqu...",2010-02-14,8.201,popular
96,284,La Garçonnière,"C.C. Baxter est employé à la Sauvegarde, grand...",1960-06-21,8.196,popular
97,324786,Tu ne tueras point,"Quand la Seconde Guerre mondiale a éclaté, Des...",2016-10-07,8.193,popular
98,490,Le Septième Sceau,"Au XIVe siècle, la grande peste ravage la Suèd...",1957-02-16,8.186,popular


In [84]:
df_recent

,id,title,overview,release_date,vote_average,category
0,986056,Thunderbolts*,"Yelena Belova, Bucky Barnes, Red Guardian, Le ...",2025-04-30,7.500,recent
1,574475,Destination finale : Bloodlines,"Stefani, 18 ans, fait d’affreux cauchemars. Da...",2025-05-14,7.200,recent
2,541671,Ballerina,"Enfant, Eve Macarro a assisté impuissante au m...",2025-06-04,7.067,recent
3,846422,The Old Guard 2,Andy et son équipe de guerriers immortels retr...,2025-07-01,6.261,recent
4,552524,Lilo & Stitch,"Sur la planète Turo, le professeur Jumba compa...",2025-05-17,7.100,recent
...,...,...,...,...,...,...
95,519182,"Moi, moche et méchant 4","Gru, Lucy et les filles, Margo, Édith et Agnès...",2024-06-20,7.042,recent
96,1079810,The Long Game,"Dans le Texas des années 1950, cinq employés, ...",2024-04-12,7.048,recent
97,1011985,Kung Fu Panda 4,Après trois aventures dans lesquelles le guerr...,2024-03-02,7.100,recent
98,1257960,सिकंदर,,2025-03-29,4.818,recent


In [85]:
df_all = pd.concat([df_popular, df_recent]).drop_duplicates(subset="id").reset_index(drop=True)

In [86]:
# Vectorisation TF-IDF sur les résumés
vectorizer = TfidfVectorizer(max_features=500, stop_words="english")
X_tfidf = vectorizer.fit_transform(df_all["overview"])

In [87]:
# Réduction dimensionnelle avec UMAP
reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, random_state=42)
embedding = reducer.fit_transform(X_tfidf.toarray())

df_all["x"] = embedding[:,0]
df_all["y"] = embedding[:,1]

/home/cor.adam/.pyenv/versions/3.10.12/envs/test-project/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



In [88]:
# Clustering KMeans (nombre de clusters à ajuster)
n_clusters = 10
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
df_all["cluster"] = kmeans.fit_predict(embedding)

In [89]:
fig = px.scatter(df_all, x="x", y="y",
                 color="cluster",
                 symbol="category",
                 hover_data=["title","release_date","vote_average","category"],
                 title="Clustering films populaires et récents TMDb")

In [90]:
fig.show()

In [100]:
import numpy as np

def find_closest_movie(df, input_title):
    # Cherche le film dans le dataset
    if input_title not in df['title'].values:
        print(f"Le film '{input_title}' n'a pas été trouvé dans le dataset.")
        return None
    
    df_recent = df[df['category'] == 'recent'].copy()
    
    # Coordonnées du film input
    input_point = df.loc[df['title'] == input_title, ['x', 'y']].values[0]
    
    # Calcul des distances euclidiennes vers tous les points
    df_recent['distance'] = np.linalg.norm(df_recent[['x', 'y']].values - input_point, axis=1)
    
    # Exclure le film lui-même (distance = 0)
    df_filtered = df_recent[df_recent['title'] != input_title]
    
    # Trouver le film avec la distance minimale
    closest_movie = df_filtered.loc[df_filtered['distance'].idxmin()]
    
    return closest_movie[['title', 'distance', 'cluster']]

In [106]:
input_title = "Interstellar"  # Exemple de film à rechercher
closest = find_closest_movie(df_all, input_title)

if closest is not None:
    print(f"Le film le plus proche de '{input_title}' est '{closest['title']}' dans le cluster {closest['cluster']} à une distance de {closest['distance']:.3f}")

Le film le plus proche de 'Interstellar' est 'M3GAN 2.0' dans le cluster 1 à une distance de 0.501
